In [ ]:
import os
import cv2
import numpy as np
import torch
import dlib
from imutils import face_utils
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from torchvision import transforms  
import torch.nn as nn
from torch.utils.data import Dataset
from albumentations.pytorch import ToTensorV2
import albumentations as A
import torch.nn.functional as F
import math
import time
from imutils import face_utils
from scipy.spatial import distance as dist
import matplotlib.pyplot as plt
import mediapipe
from BaseModels.resnetModels import BB_model
from BaseModels.efficientnetModels import BB_model
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from scipy.spatial import distance

sys.path.append("../../")
from utils import *
from BaseModels.unet_model import  UNET
# device = "cpu"


    


In [3]:

    
# Latest Model
# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Segmentation/Regression_model_1.140251636505127.pth"

# Mix Dataset
# regression_model_path = '/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/Regression_model_1.487574208665777.pth'

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModels/SpheroPipeLine/Resnet_32_IMG_SIZE__32/Regression_model_0.25639680131442016.pth"

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/Trained_Models/Resnet_32_IMG_SIZE__32/Regression_model_0.25639680131442016.pth"

# regression_model_path = "RecentModels/BioGi4eH2head/Regression_model_0.7620949026704394.pth"

# regression_model_path = "RecentModels/SpheroPipeLine/Resnet_32_IMG_SIZE__32/Regression_model_0.25639680131442016.pth"

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModelsonGpu/RecentModels/AllDataset/Regression_model_1.4074174204430023.pth"

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModelsonGpu/Recent_EffcientNetModels/EfficientNet1_6_smoothl1loss_batch_64.pt"


# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModelsonGpu/Recent_EffcientNetModels/EfficientNet1_6_smoothl1loss_batch_256.pt"


# regression_model_path = "RecentModelsonGpu/Recent_EffcientNetModels/Regression_EffcientNet__epoch_500_SmoothL1Loss_summation_batch_256_resize_64_for_gi4e_bioid_h2head_1.52.pt"

# regression_model_path = "RecentModelsonGpu/Recent_EffcientNetModels/EfficientNetWingLossV1.pt"

# regression_model_path = 'ResnetBaseWithMSEMEANwithoughtCoordConv/checkpoint.pt'


regression_model_path = 'HeatMap_Regression/HeatMapRegressionModel.pth'

REGRESSION_MODEL = load_model(
    model_path=regression_model_path)



RESIZE_AMT = 256



In [39]:
x= count_parameters(REGRESSION_MODEL)

In [41]:
x

31043521

In [4]:
# BioID
val_csv_path = "/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/MP2GAZE/AllCoordinatesMp2GazeTest.csv"


val_df = pd.read_csv(val_csv_path)



In [5]:
val_df.head()

,ImageName,Coordinates,DataSet Type
0,"MPIIGaze/Data/Original/p00/day02/0350.jpg""","[576.75131084363, 446.9430824441167, 595.62107...",MP2GAZE
1,"MPIIGaze/Data/Original/p00/day23/0201.jpg""","[629.1027244977723, 418.5422721626321, 649.159...",MP2GAZE
2,"MPIIGaze/Data/Original/p00/day22/0437.jpg""","[519.3837103984533, 379.3424773235636, 551.963...",MP2GAZE
3,"MPIIGaze/Data/Original/p00/day34/0296.jpg""","[563.4499942709884, 374.6517620194357, 591.877...",MP2GAZE
4,"MPIIGaze/Data/Original/p00/day10/0198.jpg""","[659.4630047049818, 463.58724173032533, 685.19...",MP2GAZE


In [ ]:



def prediction_image(model,image):
        
        
        model.eval()
        
        
        img = image[:,:,::-1]
        
        
        img = cv2.resize(img,(RESIZE_AMT,RESIZE_AMT))
        
        img = img/255.0
        
        img = torch.tensor(img,dtype=torch.float32).permute(2,0,1)
        
        img = img.unsqueeze(0).to(device)
        
        with torch.no_grad():
                out_coord =heatmap2argmax(model(img).view(-1,1,256,256))
        
        print()
        print(out_coord)
                
                
        
        
        
        img = img.squeeze(0)

        image = transforms.ToPILImage()
        
        
        pred_coord = out_coord.detach().cpu().numpy()[0][0]
        
        return image,pred_coord

In [8]:
def cropeyes(image, inner_array, outer_array):

    arr = {"top_left": [inner_array[0]-5, inner_array[1]-20],
           "bottom_right": [outer_array[0]+5, outer_array[1]+20]}

    return image[arr["top_left"][1]:arr["bottom_right"][1], arr["top_left"][0]:arr["bottom_right"][0]], arr

In [9]:



def rescale_coordinate(coord,original_image,resize_amt):
    
    h,w = original_image.shape[:2]
    coord[0] = int((coord[0]/resize_amt) * w)
    coord[1] = int((coord[1]/resize_amt) * h)
    
    return coord

In [10]:
def scaleCoorinatesToOriginalImage(pred_coords,eye_margin):
    
    # {'top_left': array([385, 214]), 'bottom_right': array([426, 226])}
    
    x1 = eye_margin["top_left"][0] + pred_coords[0]
    y1 = eye_margin["top_left"][1] + pred_coords[1]
    
    return [x1,y1]
    

In [11]:
def calculateEuclideanDistance(coord1,coord2):
    return (((coord1[0])-float(coord2[0]))^2  + (float(coord1[0])-float(coord2[0]))^2) ^ 0.5

In [12]:
IMG_DIR = "/home/nipun/Documents/Uni_Malta/Datasets/"

In [ ]:
maximizedNormalizedError = []
times_array = []
for i, (_, rows) in enumerate(val_df.iterrows()):

        bbox = [int(float(x)) for x in rows["Coordinates"][1:-1].split(',')]

        left_inner = bbox[0:2]
        left_center = bbox[2:4]
        left_outer = bbox[4:6]

        right_inner = bbox[6:8]
        right_center = bbox[8:10]
        right_outer = bbox[10:12]

        img_path = os.path.join(IMG_DIR, rows["ImageName"][:-1])
        # #
        img = cv2.imread(img_path)


        IPD = distance.euclidean(left_center, right_center)
        

       

        left_eye, Leye = cropeyes(img, left_inner, left_outer)
        right_eye, Reye = cropeyes(img, right_inner, right_outer)
        
        
        
        start_time = datetime.datetime.now()
        _, pred_l_eye = prediction_imageV1(
            model=REGRESSION_MODEL, image=left_eye,resize_amt=RESIZE_AMT)
        
        end_time = datetime.datetime.now()
        
        times_array.append((end_time-start_time).total_seconds())

        _, pred_r_eye = prediction_imageV1(
            model=REGRESSION_MODEL, image=right_eye,resize_amt=RESIZE_AMT)
        
        
        pred_l_eye = rescale_coordinate(pred_l_eye, left_eye, RESIZE_AMT)

        pred_r_eye = rescale_coordinate(pred_r_eye, right_eye, RESIZE_AMT)
        
        
        pred_l_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            pred_l_eye, Leye)
        pred_r_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            pred_r_eye, Reye)
        
        
        cv2.circle(img, (int(pred_l_eye_toOriginaImage[0]), int(
            pred_l_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)

        cv2.circle(img, (int(pred_r_eye_toOriginaImage[0]), int(
            pred_r_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)
        
        
        cv2.circle(img, (int(left_inner[0]), int(
            left_inner[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(left_center[0]), int(
            left_center[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(left_outer[0]), int(
            left_outer[1])), 1, (0, 0, 255), -1)

        cv2.circle(img, (int(right_inner[0]), int(
            right_inner[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(right_center[0]), int(
            right_center[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(right_outer[0]), int(
            right_outer[1])), 1, (0, 0, 255), -1)
        
         
          
        left_eye_euclidea_distance = distance.euclidean(
            pred_l_eye_toOriginaImage, left_center)

        right_eye_euclidea_distance = distance.euclidean(
            pred_r_eye_toOriginaImage, right_center)
        
        eMax = max(left_eye_euclidea_distance, right_eye_euclidea_distance)/IPD

        print(eMax)

        maximizedNormalizedError.append(eMax)
    
        # break

/tmp/ipykernel_10273/913963836.py:4: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  pts = torch.cat([index%W, index//W], dim=2)



tensor([[[ 95, 112]]], device='cuda:0')

tensor([[[112, 120]]], device='cuda:0')
0.01681064060301542

tensor([[[100, 108]]], device='cuda:0')

tensor([[[110, 110]]], device='cuda:0')
0.01499526786525223

tensor([[[128,  99]]], device='cuda:0')

tensor([[[136, 111]]], device='cuda:0')
0.014134222674039373

tensor([[[105, 119]]], device='cuda:0')

tensor([[[142, 124]]], device='cuda:0')
0.04407683212845996

tensor([[[124, 107]]], device='cuda:0')

tensor([[[136, 107]]], device='cuda:0')
0.015607862099528717

tensor([[[136,  99]]], device='cuda:0')

tensor([[[140, 108]]], device='cuda:0')
0.01563339910144465

tensor([[[126, 119]]], device='cuda:0')

tensor([[[144, 123]]], device='cuda:0')
0.0055828472590842525

tensor([[[136, 106]]], device='cuda:0')

tensor([[[116, 105]]], device='cuda:0')
0.0636665177057444

tensor([[[124, 103]]], device='cuda:0')

tensor([[[140, 105]]], device='cuda:0')
0.0310860829281539

tensor([[[130, 118]]], device='cuda:0')

tensor([[[124, 110]]], device='cuda:0'

In [34]:
def CheckForLess(list1, val):
    
    l1 = []
    for x in list1:
        
        if x <= val:
            l1.append(x)
    return l1

In [35]:
times_array

[0.043262,
 0.028022,
 0.030422,
 0.046971,
 0.028594,
 0.023727,
 0.027655,
 0.026465,
 0.028114,
 0.026544,
 0.026883,
 0.025137,
 0.026246,
 0.027059,
 0.026753,
 0.02587,
 0.026226,
 0.025693,
 0.025826,
 0.026751,
 0.023445,
 0.026438,
 0.03947,
 0.027644,
 0.026444,
 0.024697,
 0.025728,
 0.026153,
 0.026187,
 0.027228,
 0.024485,
 0.026635,
 0.027244,
 0.02536,
 0.026021,
 0.023219,
 0.025985,
 0.026525,
 0.026225,
 0.028133,
 0.025227,
 0.025945,
 0.026264,
 0.027051,
 0.023557,
 0.023789,
 0.026389,
 0.02693,
 0.024372,
 0.027405,
 0.025934,
 0.025696,
 0.027585,
 0.025679,
 0.027138,
 0.025694,
 0.024566,
 0.024992,
 0.026032,
 0.025609,
 0.024113,
 0.026636,
 0.025075,
 0.025723,
 0.026347,
 0.027111,
 0.023689,
 0.026305,
 0.026486,
 0.026191,
 0.027458,
 0.024047,
 0.027768,
 0.025479,
 0.027397,
 0.027315,
 0.025814,
 0.025592,
 0.026707,
 0.02787,
 0.023927,
 0.027581,
 0.025096,
 0.024976,
 0.02567,
 0.026029,
 0.026666,
 0.026585,
 0.027547,
 0.024179,
 0.025194,
 0.02

In [21]:
e_0_25 = CheckForLess(maximizedNormalizedError,0.25)

e_0_05 = CheckForLess(maximizedNormalizedError,0.05)

e_0_1 = CheckForLess(maximizedNormalizedError,0.1)

In [22]:
(len(e_0_25)/len(maximizedNormalizedError)) *100

100.0

In [23]:
(len(e_0_05)/len(maximizedNormalizedError)) * 100

100.0

In [24]:
len(e_0_1)/len((maximizedNormalizedError)) * 100

88.28125

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(img[:,:,::-1])


In [ ]:
plt.imshow(right_eye)